In [118]:
import pandas as pd
import numpy as np

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.image as mpimg

# Préparation des données
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Modèles
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

# Option d'affchage
pd.set_option('display.max_columns', None)


from datetime import datetime


In [119]:
# Importation de la base des données calculées

df0 = pd.read_csv('data/bdd_complete2.csv', sep = ',', encoding = 'latin-1')

# Copie de la base

df = df0.copy()

# 0. Processing 

In [120]:
df = df[[
       'Date', 'Nature.de.séance',
       'Réplique', 'Didascalie',
       'Président.de.séance_propre', 'nom.famille', 'sexe', 'age',
       'groupe.sigle', 'commissions', 'nb.mandats', 'cabcollab', 'duree.pol',
       'clustRFSP', 'clustVEP', 'hautdip', 'majo',
       'profsigni2', 'Groupe'
        ]]

In [121]:
df['Date'] = df.apply(lambda row: (datetime.strptime(row.Date, '%Y-%m-%d') - datetime(2015, 1, 1, 0, 0)).days, axis = 1)

In [122]:
df['Didascalie'] = df.apply(lambda row: 0 if str(row.Didascalie) == 'nan' else 1, axis =  1)

In [123]:
df['sexe'] = df.apply(lambda row: 0 if row.sexe == 'F' else 1, axis =  1)

In [124]:
df['majo'] = df.apply(lambda row: 1 if row.majo else 0, axis =  1)

In [125]:
df['Groupe'] = df.apply(lambda row: 1 if row.Groupe == 'Exp' else 0, axis =  1)

In [126]:
df['cabcollab'] = df.apply(lambda row: 1 if row.cabcollab else 0, axis =  1)

In [127]:
df.sample(10)

,Date,Nature.de.séance,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,majo,profsigni2,Groupe
310084,936,Confiance dans la vie publique,Cest pourtant un bon amendement.,0,Présidence de Mme Carole Bureau-Bonnard,Le Fur,1,61,LR,Finances,4,1,27,5 - Dép. sortant,5 - Dép. sortant,ENA,0,cadsuppub,1
491311,1405,Questions au Gouvernement,Cest une erreur<U+0080>!,0,Présidence de M. Richard Ferrand,Descoeur,1,55,LR,DvptDurable,1,0,29,5 - Dép. sortant,5 - Dép. sortant,Master,0,cadsuppub,1
72995,1511,Questions au Gouvernement,Tout dabord est-il exact que le Gouvernement...,0,Présidence de M. Richard Ferrand,Courson (de),1,65,LC,Finances,5,1,32,5 - Dép. sortant,5 - Dép. sortant,ENA,1,cadsuppub,1
12911,1512,Référendums dinitiative citoyenne,Le peuple protège les minorités bien plus que...,0,Présidence de M. Hugues Renson,Corbière,1,49,LFI,Defense,0,0,14,3 - Ancr. local,3 - Ancr. local,Master,0,cadsuppub,1
679,1030,Projet de loi de financement de la sécurité so...,Tandis que les fraudeurs aux prestations socia...,0,Présidence de M. Sylvain Waserman,Quatennens,1,27,LFI,Aff. Sociales,0,0,0,1 - Soc. civile,6  Exp. Pol.,Pro,0,EmployeInter,1
119195,1231,Lutte contre les violences sexuelles et sexistes,Tous les actes de cette nature ne seront pas s...,0,Présidence de M. François de Rugy,Fajgeles,0,47,LREM,Lois,0,0,9,2 - 2de carrière,2 - 2de carrière,Master,1,cadsuppub,1
368044,1869,Gel des matchs de football le 5 mai,Jai encore le cur noué car je repense à ce ...,0,Présidence de M. Sylvain Waserman,Colombani,1,50,NI,DvptDurable,0,0,2,1 - Soc. civile,6  Exp. Pol.,Doct,0,proflib,1
272401,1196,Nouveau pacte ferroviaire,Ces trains de nuit ont été supprimés pour des ...,1,Présidence de M. François de Rugy,Dubié,0,59,NI,Aff. Sociales,1,0,9,5 - Dép. sortant,5 - Dép. sortant,Pro,0,cadsuppub,1
358590,1954,Prorogation de létat durgence sanitaire,La quatorzaine est prévue pour protéger les po...,0,Présidence de Mme Laetitia Saint-Paul,Véran,1,37,LREM,Aff. Sociales,1,0,5,5 - Dép. sortant,5 - Dép. sortant,Doct,1,proflib,1
440023,1113,Indivision successorale et politique du logeme...,rapporteur. Cest effectivement un amendement...,0,Présidence de M. Marc Le Fur,Letchimy,1,64,NG,affaires économiques,2,0,25,5 - Dép. sortant,5 - Dép. sortant,Doct,0,cadsuppub,1


# I. Préparation des données

In [128]:
# Standardisation des données

min_max_scaler = preprocessing.MinMaxScaler()
df[['Date', 'age'
    ]] = min_max_scaler.fit_transform(df[['Date', 'age']])

In [129]:
def one_hot_encode(dataframe, variable):
    
    dataframe[variable] = dataframe[variable].astype(str)


    
    le = LabelEncoder()
    
    var = variable + '_encoded'
    
    dataframe[var] = le.fit_transform(dataframe[variable])

    ohe = OneHotEncoder()

    X = ohe.fit_transform(dataframe[var].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = [variable + str(int(i)) for i in range(X.shape[1])])
    dataframe = pd.concat([dataframe, dfOneHot], axis=1).drop(columns = [variable, var]).dropna()
    
    return dataframe.copy()

In [130]:
df = one_hot_encode(df, 'Nature.de.séance')
df = one_hot_encode(df, 'nom.famille')
df = one_hot_encode(df, 'groupe.sigle')
df = one_hot_encode(df, 'commissions')
df = one_hot_encode(df, 'clustRFSP')
df = one_hot_encode(df, 'clustVEP')
df = one_hot_encode(df, 'hautdip')
df = one_hot_encode(df, 'profsigni2')
df = one_hot_encode(df, 'Président.de.séance_propre')

In [136]:
df.sample(10)

Date                                           Réplique  \
94166   0.575480   Je comprends très bien la réponse de Mme la m...   
185831  0.300091                La parole est à M. Olivier Falorni.   
78855   0.005489  Quant à votre projet de suppression de la taxe...   
24883   0.762123                                Exactement<U+0080>!   
277613  0.102470   rapporteur général. Nous avons calculé lévol...   
36923   0.576395   Ma question  à laquelle jassocie mes collègu...   
208097  0.301006   Quand la betterave à sucre sest développée d...   
411298  0.546203                    La parole est à M. le ministre.   
136444  0.596523  Que croyez-vous quil se passe quand toute lo...   
246966  0.824337   Ça marche ensemble<U+0080>! Dites merci aux g...   

        Didascalie  sexe       age  nb.mandats  cabcollab  duree.pol  majo  \
94166          0.0   1.0  0.636364         0.0        0.0       22.0   1.0   
185831         0.0   1.0  0.363636         2.0        1.0       19.0   1.0   
78855          1.0   1.0  0.618182         5.0        0.0       22.0   0.0   
24883          0.0   0.0  0.672727         1.0        0.0       22.0   0.0   
277613         0.0   1.0  0.618182         3.0        0.0       28.0   1.0   
36923          0.0   0.0  0.581818         0.0        0.0        9.0   1.0   
208097         0.0   1.0  0.745455         4.0        0.0       33.0   0.0   
411298         0.0   1.0  0.563636         1.0        1.0       22.0   1.0   
136444         0.0   1.0  0.636364         0.0        0.0        7.0   0.0   
246966         0.0   1.0  0.763636         0.0        0.0       33.0   0.0   

        Groupe  Nature.de.séance0  Nature.de.séance1  Nature.de.séance2  \
94166      1.0                0.0                0.0                0.0   
185831     1.0                0.0                0.0                0.0   
78855      1.0                0.0                0.0                0.0   
24883      1.0                0.0                0.0                0.0   
277613     1.0                0.0                0.0                0.0   
36923      1.0                0.0                0.0                0.0   
208097     1.0                0.0                0.0                0.0   
411298     1.0                0.0                0.0                0.0   
136444     1.0                0.0                0.0                0.0   
246966     1.0                0.0                0.0                0.0   

        Nature.de.séance3  Nature.de.séance4  Nature.de.séance5  \
94166                 0.0                0.0                0.0   
185831                0.0                0.0                0.0   
78855                 0.0                0.0                0.0   
24883                 0.0                0.0                0.0   
277613                0.0                0.0                0.0   
36923                 0.0                0.0                0.0   
208097                0.0                0.0                0.0   
411298                0.0                0.0                0.0   
136444                0.0                0.0                0.0   
246966                0.0                0.0                0.0   

        Nature.de.séance6  Nature.de.séance7  Nature.de.séance8  \
94166                 0.0                0.0                0.0   
185831                0.0                0.0                0.0   
78855                 0.0                0.0                0.0   
24883                 0.0                0.0                0.0   
277613                0.0                0.0                0.0   
36923                 0.0                0.0                0.0   
208097                0.0                0.0                0.0   
411298                0.0                0.0                0.0   
136444                0.0                0.0                0.0   
246966                0.0                0.0                0.0   

        Nature.de.séance9  Nature.de.séance10  Nature.de.séance11  \


In [137]:
df.to_csv('data/one_hot_encoded_data.csv')

In [138]:
# On définit la variable objectif

df['Réplique'] = df.apply(lambda row : len(row.Réplique), axis = 1)

In [140]:
y = df[['Réplique']]
X = df.drop(columns = ['Réplique'])

# On choisit un échantillon de validation de 20 %

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [145]:
N = X.shape[1]

# II. Modélisation

In [141]:
# Affichage de la matrice de corrélation

fig, ax = plt.subplots(figsize = (13, 10)) 
fig = sns.heatmap(df.sample(100).corr(), cmap= 'coolwarm', annot = True)

Error in callback <function flush_figures at 0x7fc3058f89d0> (for post_execute):


KeyboardInterrupt: 

## II.1 Regressions linéaires et polynomiales

### Regression linéaire multiple

On commence par une regression linéaire pour avoir une première idée de l'influence des variables sur le prix

In [142]:
lin_reg = LinearRegression().fit(X_train, y_train)

# On prédit à partir de l'échantillon de test pour calculer les scores

y_pred = lin_reg.predict(X_test)

KeyboardInterrupt: 

In [ ]:
px.histogram(pd.DataFrame([
    (X.columns[i], lin_reg.coef_[0][i]) for i in range(47)
                    ]).T.rename(index = {0 : 'variable', 1 : 'coeff'}).T, x = 'variable', y = 'coeff', histfunc = 'sum'
            ).show()

print('MSE :', mean_squared_error(y_test, y_pred))

In [ ]:
# Affichage des p-values

mod = sm.OLS(y,X)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']
pd.DataFrame(p_values).T

On observe que :  

- les variables avec les plus petites p-values sont les score aux commerces et score_monument
- être proche des commerces atypiques mais loin des commerces en général ferait monter le prix
- la variable qui semble être la plus significative est score_monument (p-value la plus basse sans compter les TypeVoie, et coefficient le plus grand)
- score_metro a étonnement un coefficient négatif
- Les p-values concernant TypeVoie ne semblent pas pertinentes, on manque sans doute de données pour certaines modalités

### Regressions polynomiales

Il est très probable que les variables n'aient pas un effet linéaire sur le prix au m<sup>2<sup>.
    
C'est pourquoi on choisit de faire des regressions polynomiales, en augmentant progressivement le degré, et en s'arrêtant dès qu'on observe de l'overfitting sur l'échantillon de test.

In [ ]:
def poly_reg(n):
    """
    Cette fonction effectue une regression polynomiale sur la base d'entrainement et affiche 
    la MSE pour l'echantillon de test
    """
    
    # Agrandissement de la base d'entrainement avec l'ajout des degrés successifs
    poly_X = X_train.copy()
    
    for i in range(2, n+1):
        
        index = [str(j) + '^' + str(i) for j in X.columns[:32]]
        poly_X[[str(j) + '^' + str(i) for j in X.columns[:32]]] = poly_X[[j for j in X.columns[:32]]].pow(i)
    
    poly_reg = LinearRegression().fit(poly_X, y_train)
    
    # Agrandissement de la base d'de test avec l'ajout des degrés successifs
    poly_X_test = X_test.copy()
    
    for i in range(2, n+1):
        
        index = [str(j) + '^' + str(i) for j in X.columns[:32]]
        poly_X_test[[str(j) + '^' + str(i) for j in X.columns[:32]]] = poly_X_test[[j for j in X.columns[:32]]].pow(i)
    
    # Regression
    y_pred = poly_reg.predict(poly_X_test)
    MSE = mean_squared_error(y_test, y_pred)
    
    # Affichage des coefficients pour le degré 2
    fig = px.histogram(pd.DataFrame([
        (poly_X.columns[i], poly_reg.coef_[0][i]) for i in range(28 + 32 * (n - 1))
                            ]).T.rename(
        index = {0 : 'variable', 1 : 'coeff'}
                                        ).T.sort_values(by = 'variable'), x = 'variable', y = 'coeff') 
    
    if n == 2:
        
        fig.show()
        
    else:
        
        print('degré : ', n)

    return MSE

In [ ]:
# Regressions jusqu'à l'overfitting (MSE > 0.4 sur l'échantillon test)

i = 2
val = poly_reg(2)
list_mse_degree = [val]


while val < .016:
    i += 1
    val = poly_reg(i)
    list_mse_degree.append(val) 
    
    if i > 25:
        break
    

px.line(x = [i + 2 for i in range(len(list_mse_degree))], y = list_mse_degree).show()

On observe que :  
- le degré à partir duquel on observe de l'overfitting dépend beaucoup de l'échantillon de test, qui est choisi au hasard. On a pu observer de l'overfitting au degré 25 comme au degré 8 avec des échantillons différents
- Dans la regression de degré 2, pour presque chaque variable, le coefficient de la variable et de la variable au carré ont un exposant de signe opposé. Cela semble signifier que le premier coeffient "compense" l'effet du deuxième, et donc que l'effet de chaque variable est plus complexe qu'un effet linéaire
- Seul score_monument a significativement deux coefficients positif. Son effet semble clair : plus on est proche des monuments, plus le prix est haut

In [ ]:
# Pour obtenir de nouveaux échantillons si échec :  
y = df[['prixm2']]
X = df.drop(columns = ['prixm2'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

## II.2 Regression pénalisée

Etant donnée le grand nombre de variables dont nous disposons, il nous parrait cohérent d'utiliser une regression pénalisée afin de sélectionner les variables les plus significatives.
De plus, certaines de nos variables sont corrélées entre elles.

Ici, on a fait baisser l1_ratio progressivement jusqu'à obtenir 10 variables dont les coefficients sont non nul.

In [232]:
df1 = df.copy()

y = df1[['Réplique']]
X = df1.drop(columns = ['Réplique'])

# On choisit un échantillon de validation de 20 %

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [233]:
EN_reg = ElasticNet(alpha=.3, copy_X=False, fit_intercept = False, l1_ratio=1)

EN_reg.fit(X_train, y_train)

ElasticNet(alpha=0.3, copy_X=False, fit_intercept=False, l1_ratio=1)

In [234]:
y_pred = EN_reg.predict(X_test)
print('MSE : ', mean_squared_error(y_test, y_pred))

MSE :  52348.48226013714


In [235]:
pd.DataFrame([(X.columns[i], EN_reg.coef_[i]) for i in range(N)]).T.rename(index = {0 : 'variable', 1 : 'coeff'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

In [236]:
result = [(EN_reg.coef_[i], X.columns[i]) for i in range(N) if abs(EN_reg.coef_[i]) > 50]

In [237]:
sorted(result)

[(-137.1449751829627, 'nom.famille443'),
 (-102.14269241605304, 'nom.famille446'),
 (-94.54443331887184, 'nom.famille166'),
 (-92.8511770130681, 'nom.famille319'),
 (-91.41752579055155, 'nom.famille442'),
 (-77.98972308352326, 'Nature.de.séance487'),
 (-75.45943687608731, 'nom.famille172'),
 (-72.17897578570624, 'Nature.de.séance386'),
 (-64.64507300742152, 'nom.famille413'),
 (-61.53515582872654, 'nom.famille287'),
 (-57.861909401329115, 'nom.famille248'),
 (-51.81054935101797, 'nom.famille28'),
 (-50.22120891974088, 'Nature.de.séance192'),
 (74.03068308452298, 'Nature.de.séance387'),
 (103.42934023715745, 'Didascalie'),
 (227.9863074346519, 'clustRFSP1'),
 (231.1742855789275, 'clustRFSP3'),
 (245.28059702341693, 'clustRFSP4'),
 (247.35080376686213, 'clustRFSP2'),
 (251.0371425556208, 'clustRFSP0')]

## II.3 Regression par Random Forest

Dans cette partie, nous allons faire des regressions grâce aux arbres de décision. 
Nous allons rechercher les meilleurs paramètres pour notre regression.
Pour cela, on teste un à un les paramètres suivants :  
- profondeur des arbres  
- nombre d'arbres  
- minimun d'exemples requis pour splitter l'arbre  
- nombre de feuilles maximum par arbre  

In [ ]:
# On définit d'abord une fonction qui donne l'importance des variables vis à vis d'un certain modèle

def feat_importance(model, x_train, y_train, X):
    """
    Renvoie le tableau de l'importance des variables vis à vis du modèle par la méthode des permutations
    """

    result = permutation_importance(
                                    model, 
                                    X, 
                                    y, 
                                    n_repeats = 3,
                                    random_state = 0
                                    )['importances_mean']
    
    importance = pd.DataFrame(result, index = X.columns, columns = ["Importance"])
    
    return importance.sort_values(by = ['Importance'], ascending = False)

In [ ]:
# Création des essembles de tests et d'entrainement, on choisit une taile de test de 30% ici

X = df.drop(['prixm2'], axis = 1)
x = np.array(X)
y = np.array(df['prixm2'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 41)
# random_state correspond à la graine générant l'échantillon aléatoire

### II.3.1 Profondeur maximale

In [ ]:
max_depth_ls = [1, 10, 13, 15, 17, 20, 25, 30] # profondeurs maximales des arbres de décision testées
mse_train_max_depth = []
mse_test_max_depth = []

# Pour chaque profondeur max, on regresse avec random forest

for m in max_depth_ls :
    
    print('Profondeur téstée : ', m)
    
    rf = RandomForestRegressor(
                            max_depth = m, 
                            random_state=0,
                            n_estimators = 30) # nombre d'arbres utilisés
    
    rf = rf.fit(x_train, y_train)
    y_pred_train = rf.predict(x_train)
    y_pred = rf.predict(x_test)
    
    mse_train_max_depth.append(mean_squared_error(y_train, y_pred_train))
    mse_test_max_depth.append(mean_squared_error(y_test, y_pred))

In [ ]:
# On affiche ensuite les performances de la regression sur les deux échantillon (train et test)

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(max_depth_ls, mse_train_max_depth, color = 'red', label = 'Train')
plt.plot(max_depth_ls, mse_test_max_depth, color = 'blue', label = 'Test')
plt.title('MSE en fonction de max_depth')
plt.legend()
plt.show()

On observe que l'overfitting débute après max_depth > 15 (le score sur l'échantillon d'entrainement continue de decroitre alors qu'il commence à croitre sur l'échantillon de test)

In [ ]:
#On regarde la valeur qui minimise la MSE sur l'ensemble de test

max_depth_ls[mse_test_max_depth.index(min(mse_test_max_depth))]

**On garde donc max_depth = 15 pour la suite**

### II.3.2 Nombre d'arbres

In [ ]:
# On regarde maintenant l'effet du nombre d'arbre sur l'effet de la regression

nb_estimators_ls = [1, 2, 3, 5, 20, 40, 50, 60, 80]
mse_train_nb_estimators = []
mse_test_nb_estimators = []

for m in nb_estimators_ls :
    print("Nombre d'arbres testés : ", m)
    rf = RandomForestRegressor(max_depth = 15, 
                               random_state = 0,
                                n_estimators = m)    
    
    rf = rf.fit(x_train, y_train)
    y_pred_train = rf.predict(x_train)
    y_pred = rf.predict(x_test)
    
    mse_train_nb_estimators.append(mean_squared_error(y_train, y_pred_train))
    mse_test_nb_estimators.append(mean_squared_error(y_test, y_pred))


In [ ]:
# On affiche ensuite les performances de la regression sur les deux échantillon (train et test)

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(nb_estimators_ls, mse_train_nb_estimators, color = 'red', label = 'Train')
plt.plot(nb_estimators_ls, mse_test_nb_estimators, color = 'blue', label = 'Test')
plt.title('MSE en fonction de n_estimators')
plt.legend()
plt.show()

**La courbe semble strictement décroissante, on doit arbitrer entre complexité algorithmique et performance.  
On garde n_estimators = 60 pour la suite**

### II.3.3 Minimun d'exemples requis pour splitter l'arbre  

In [ ]:
# On fait varier le nombre minimum d'exemple requis pour créer une feuille/noeud

samples_leaf_ls = [1, 2, 3, 4, 10]
mse_train_samples_leaf = []
mse_test_samples_leaf = []


for m in samples_leaf_ls :
    print('min_samples_leaf testé : ', m)
    rf = RandomForestRegressor( max_depth = 15, 
                                min_samples_leaf = m,
                                n_estimators = 60, 
                                random_state = 0
                              )    
    
    rf = rf.fit(x_train, y_train)
    y_pred_train = rf.predict(x_train)
    y_pred = rf.predict(x_test)
    
    mse_train_samples_leaf.append(mean_squared_error(y_train, y_pred_train))
    mse_test_samples_leaf.append(mean_squared_error(y_test, y_pred))

In [ ]:
# On affiche ensuite les performances de la regression sur les deux échantillon (train et test)

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(samples_leaf_ls, mse_train_samples_leaf, color='red', label='Train')
plt.plot(samples_leaf_ls, mse_test_samples_leaf, color='blue', label='Test')
plt.title('MSE en fct de min samples leaf')
plt.legend()
plt.show()

**On garde donc min_samples_leaf = 1 pour la suite**

### II.3.4 Nombre maximum de feuilles par arbres

In [ ]:
max_leaf_ls = [2, 10, 100, 150, 200, 1000, 1500]
mse_train_max_leaf = []
mse_test_max_leaf = []



for m in max_leaf_ls :
    
    print('Nombre de feuilles max testé : ', m)
    rf = RandomForestRegressor(max_depth = 15, 
                               min_samples_leaf = 1, 
                               max_leaf_nodes = m,
                               n_estimators = 60)   
    
    rf = rf.fit(x_train, y_train)
    y_pred_train = rf.predict(x_train)
    y_pred = rf.predict(x_test)
    mse_train_max_leaf.append(mean_squared_error(y_train, y_pred_train))
    mse_test_max_leaf.append(mean_squared_error(y_test, y_pred))

In [ ]:
# On affiche ensuite les performances de la regression sur les deux échantillon (train et test)

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(max_leaf_ls, mse_train_max_leaf, color = 'red', label = 'Train')
plt.plot(max_leaf_ls, mse_test_max_leaf, color = 'blue', label = 'Test')
plt.title('MSE en fonction max_leaf_nodes')
plt.legend()
plt.show()

**La courbe semble strictement décroissante, on doit arbitrer entre complexité algorithmique et performance.  
On garde max_leaf_nodes = 1000 pour la suite**

In [ ]:
# On a maintenant tous nos paramètres

rf = RandomForestRegressor(
                        max_depth = 15, 
                        min_samples_leaf = 1, 
                        max_leaf_nodes = 1000,
                        n_estimators = 60
                            )    

rf = rf.fit(x_train, y_train)
y_pred_train = rf.predict(x_train)
y_pred = rf.predict(x_test)
oo = np.zeros(y_pred.shape)

print('MSE train : ', mean_squared_error(y_train, y_pred_train))
print('MSE test : ', mean_squared_error(y_test, y_pred))
print('MSE modèle nulle : ', mean_squared_error(y_test, oo))

importance = feat_importance(rf, x_train, y_train, X)

In [ ]:
importance.plot(kind = 'barh', figsize = (18, 14))

In [ ]:
print('Rappel des variables retenue par ElasticNet : ', signif_EN)

- On retrouve la présence de score_monument comme variable la plus importante, et on retrouve globalement les mêmes variables importantes.  
- On retrouve que la longitude est plus importante que la latitude.  
- On note aussi que les TypeVoie les plus importantes sont les 1, 2 et 11 (Avenue, Boulevard, Rue). 
- Par contre, Quartier surpasse Arrondissement de beaucoup, comme ces deux variables sont très corrélées, cela ne pose pas de problème de cohérence.
- NbPieces, et les scores pour les jardins, le metro et les commerce sont plus iportants ici.


In [ ]:
# On garde que les variables raisonnablement significatives (>0.01)

X = df[['score_monument', 'lon', 'lat', 'NbPieces', 'score_jardin', 'score_metro']]

x = np.array(X)
y = np.array(df['prixm2'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 41)

rf = RandomForestRegressor(max_depth = 15, random_state = 0, min_samples_leaf = 1, max_leaf_nodes = 1000,
                                 n_estimators = 60)
rf = rf.fit(x_train,y_train)


y_pred = rf.predict(x_test)
y_pred_train= rf.predict(x_train)

oo = np.zeros(y_pred.shape)

print('MSE train : ', mean_squared_error(y_train, y_pred_train))
print('MSE test : ', mean_squared_error(y_test, y_pred))
print('MSE model nulle : ', mean_squared_error(y_test, oo))

importance = feat_importance(rf, x_train, x_test, X)
importance.plot(kind='barh')

On retrouve exactement les mêmes résultats, sauf que la latitude a dépassé la longitude.

## II.4 Réseau de neurones

Les réseaux de neurones sont des architectures qui permettent de rendre compte des influences complexes des variables sur le prix.

In [ ]:
# On définit les échantillons

y = df[['prixm2']]
X = df.drop(columns = ['prixm2'])

Après plusieurs essais, un réseau 28 - 2 - 3 - 1 fournit des résultats satisfaisant. 

In [ ]:
# Construction de l'architecture du réseau

model = Sequential()

model.add(Dense(70, input_dim = 47, activation = 'relu'))

model.add(Dense(12))
model.add(Dense(13))
model.add(Dense(15))
model.add(Dense(5))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')

In [ ]:
# Lancement de la phase d'apprentissage

history = model.fit(X, y, validation_split = 0.2,  epochs = 30)

In [ ]:
# On affiche l'évolution de la loss au fil des époques pour les échantillons train et test.

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(history.history['loss'])
plt.title('Evolution de MSE sur X_train au fil des époques')
plt.ylabel('mse')
plt.legend(['train'], loc = 'upper left')
plt.show()

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(history.history['val_loss'])
plt.title('Evolution de MSE sur X_test au fil des époques')
plt.ylabel('mse')
plt.legend(['test'], loc = 'upper left')
plt.show()

 **En prenant en compte les variables sélectionnées par ElsaticNet précédemment**

In [ ]:
X_significatif = X[signif_EN]

In [ ]:
# On reprend la même architecture en changeant le nombre de neurones d'entrée.

model_significatif = Sequential()

model_significatif.add(Dense(2, input_dim = 10, activation='relu'))
model_significatif.add(Dense(3))
model_significatif.add(Dense(1))

model_significatif.compile(optimizer = 'adam', loss = 'mse')

In [ ]:
# Lancement de la phase d'apprentissage

history_significatif = model_significatif.fit(X_significatif, y, validation_split = 0.2,  epochs = 30)

In [ ]:
# On affiche l'évolution de la loss au fil des époques pour les échantillons 
# train et test en comparant avec l'autre réseau.

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(np.array(history_significatif.history['loss']))
plt.title('Evolution de MSE sur X_train au fil des époques')
plt.plot(history.history['loss'])
plt.ylabel('mse')
plt.legend(['train significatif', 'train'], loc='upper left')
plt.show()

fig, ax = plt.subplots(figsize = (18, 8))
plt.plot(history_significatif.history['val_loss'])
plt.plot(history.history['val_loss'])
plt.title('Evolution de MSE sur X_test au fil des époques')
plt.ylabel('mse')
plt.legend(['test_significatif', 'test'], loc='upper left')
plt.show()

Observations :  
- le comportement de la loss dépend de la sélection de l'échantillon de test. On a pu avoir des modèles qui apprenaient bien (décroissance de la loss dans les 4 graphiques précedents), comme des modèles qui apprenaient mal.
- La plupart du temps, on observe quand même un apprentissage qui fonctionne. Mais les performance des deux réseaux sont après 15 époques similaires (l'apprentissage avec plus de variable est même plus efficace avec peu d'époques)

In [ ]:
#Graphiques obtenus dans une bonne situation

fig, ax = plt.subplots(figsize = (18, 8))
img = mpimg.imread('img/img0.png')
imgplot = plt.imshow(img)
plt.show()

fig, ax = plt.subplots(figsize = (18, 8))
img = mpimg.imread('img/img1.png')
imgplot = plt.imshow(img)
plt.show()

**Importance des paramètres (méthode des permutations)**

In [ ]:
perm = PermutationImportance(model, scoring = 'r2', random_state = 1).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

In [ ]:
eli5.show_weights(perm, feature_names = X.columns.tolist(), top = 47)

On constate ici en général l'importance du score_monument, largement les plus important selon ce modèle (poids 10 fois supérieur aux autres variables).
On retrouve aussi l'importance du Quartier et de la longitude.

# III. Clustering

Dans cette partie, nous voulons classer les appartements en différents cluster, afin d'identifier des appartements "types", regroupant les mêmes caractéristiques classiques.  
Nous procédons alors à une ACP pour réduire la dimension de nos données, puis nous utilisons la méthode des k moyennes pour le clustering. 
On pondère chaque variable par son importance donnée dans le réseau de neurones précedemment.

In [ ]:
df_ACP = df[signif_EN].copy()

In [ ]:
df_ACP['Arrondissement'] *= 0.0045
df_ACP['Quartier'] *= 0.0167
df_ACP['score_monument'] *= 0.0555
df_ACP['TypeVoie_1'] *= 0.0030
df_ACP['lat'] *= 0.0003
df_ACP['lon'] *= 0.0087
df_ACP['score_commerce_lux'] *= 0.0004
df_ACP['TypeVoie_11'] *= 0.0003
df_ACP['periode_construction'] *= 0.0004
df_ACP['TypeVoie_2'] *= 0.0008

In [ ]:
# On réduit sur 3 variables

pca = PCA(n_components = 3)
pca.fit(df_ACP)

transformed_df = pd.DataFrame(pca.transform(df_ACP)).rename(columns = {
                            0 : 'var0', 
                            1 : 'var1', 
                            2 : 'var2',
                         })

In [ ]:
px.scatter_3d(transformed_df.sample(1000), x = 'var0', y = 'var1', z = 'var2', opacity = 1)

On peut retrouver dans cette représentation une "hélice", qui fait penser aux arrondissements de Paris disposés en escargot. Cela suggère une forte composante géographique.

In [ ]:
# On distinguera alors 6 clusters.

kmeans = KMeans(n_clusters = 6).fit(transformed_df)

In [ ]:
# On labélise

transformed_df['cluster'] = kmeans.labels_

In [ ]:
px.scatter_3d(transformed_df.sample(1000), x = 'var0', y = 'var1', z = 'var2', color = 'cluster', opacity = .7)

Les clusters ont bien été faits.

**On affiche ensuite les différentes caractéristiques des clusters obtenus.**

In [ ]:
df_cluster0 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 0].index)]]
df_cluster1 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 1].index)]]
df_cluster2 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 2].index)]]
df_cluster3 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 3].index)]]
df_cluster4 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 4].index)]]
df_cluster5 = df_ACP.iloc[[i for i in list(transformed_df[transformed_df['cluster'] == 5].index)]]

In [ ]:
display(df_cluster0.describe().T[['mean', 'std']].T)
display(df_cluster1.describe().T[['mean', 'std']].T)
display(df_cluster2.describe().T[['mean', 'std']].T)
display(df_cluster3.describe().T[['mean', 'std']].T)
display(df_cluster4.describe().T[['mean', 'std']].T)
display(df_cluster5.describe().T[['mean', 'std']].T)

On affiche le résultat dans un plan.

In [ ]:
df_ACP['cluster'] = pd.Series(kmeans.labels_)
df_ACP[['true_lat', 'true_lon']] = df0[['lat', 'lon']]

px.scatter(df_ACP, x = 'true_lon', y = 'true_lat', color = 'cluster', template = 'none', opacity = .3, color_continuous_scale = 'Jet')

On a en effet un clustering très géographique. On peut essayer d'interpréter les clusters :   
- Quartiers populaires (19ème, 20ème, une partie du 18ème et l'extrême sud)
- Quartiers familiaux (12ème, 13ème, 14ème, 15ème)
- Quartiers riche 1 (16ème)
- Quartier riche 2 (7ème et 16ème)
- Quartiers historiques (centre)
- Quartiers qui s'embourgeoisent (autour des Quartiers historiques de Paris centre, 9ème, 10ème, 11ème)

# Conclusion

On obtient des modèles dont les performances sont représentées ci-dessous.

In [ ]:
px.histogram(x = ['Reg. Lin.', 'Reg. poly.', 'ElasticNet', 'Random Forest', 'Réseau de neurones'], 
             y = [0.014, 0.013, 0.018, 0.012, 0.012], 
             range_y = [.01,.018],
             labels = {'x' : 'Modèle', 'y' : 'MSE'},
             template = 'none')

On constate la supériorité des réseaux de neurones et des regressions par Ramdom Forest, avec un avantage pour ces dernières car elles semblent plus robustes.

Au niveau de l'analyse des variables les plus importantes, on retrouve au fil des modèles :
- une omniprésence de score_monument
- une supériorité d'importance de lon sur lat
- une grande importance du Quartier ou de l'Arrondissement
- un effet positif du score_commerce_lux et négatif du score_commerce